# Handling Missing Data

### Trade-Offs

- There are multiple schemes to indicate missing data in a table or DataFrame. They use one of two strategies: using a *mask* that globally indicates missing values, or choosing a *sentinel value* that indicates a missing entry.

- A mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

- A sentinel value could be a data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number).

- Using a separate mask array requires allocation of an additional Boolean array; a sentinel value reduces the range of valid values that can be represented and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

### Missing Data in Pandas

- Pandas is constrained by its reliance on NumPy, which does not have a built-in notion of NA values for non-floating-point data types.

- Pandas could have specified bit patterns for each individual data type to indicate nullness, but NumPy supports (for example) *fourteen* basic integer types once you account for available precisions, signedness, and endianness of the encoding.

- NumPy supports masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad." Pandas could have used this approach, but it used too much overhead.

- Therefore Pandas chose to use sentinels for missing data and to use two already-existing Python null values: the special floating-point ``NaN`` value, and the Python ``None`` object.

### ``None``: Pythonic missing data

- ``None`` is a singleton object that is often used for missing data in Python code. Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

- While an object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead.

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
48.4 ms ± 953 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.45 ms ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



- The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### ``NaN``: Missing numerical data

- ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE format.

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

- Be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches. Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

- NumPy provides some special aggregations that will ignore these missing values:

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

### NaN and None in Pandas

- Pandas is built to handle both ``NaN`` and ``None`` interchangeably, converting between them where appropriate.

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

- For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA.

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

- Upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

### Operating on Null Values

- There are several methods for detecting, removing, and replacing null values in Pandas data structures:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

### Detecting null values
- Pandas data structures have two methods for detecting null data: ``isnull()`` and ``notnull()``. Either one will return a Boolean mask over the data.

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

- Boolean masks can be used directly as a ``Series`` or ``DataFrame`` index:

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

- ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

### Dropping null values

- More convenience methods are available:
    - ``dropna()`` (which removes NA values)
    - ``fillna()`` (which fills in NA values)
- For a ``Series``, the result is straightforward.

In [16]:
data.dropna()

0        1
2    hello
dtype: object

-``DataFrame``s have more options.

- We cannot drop single values from a ``DataFrame`` - only full rows or full columns. Depending on the application, you might want one or the other, so ``dropna()`` gives a number of options.

- By default, ``dropna()`` will drop all rows in which *any* null value is present:

In [18]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [19]:
df.dropna()

,0,1,2
1,2.0,3.0,5


- Or you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value.

In [20]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


- This drops good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values. This can be specified through the ``how`` or ``thresh`` parameters.

- The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
- You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [21]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [22]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


- The ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [23]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


- The first and last row have been dropped because they contain only two non-null values.

### Filling null values

- Sometimes you'd rather replace NA values with a valid value. This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.

- You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

In [24]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

- Fill NA entries with a single value, such as zero:

In [25]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

- Forward-fill to propagate the previous value forward:

In [26]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

- Back-fill to propagate the next values backward:

In [27]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

- We can also specify an ``axis`` along which the fills take place in a ``DataFrame``.

In [28]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [29]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
